In [2]:
import pandas as pd
import sqlite3
import os
import glob
import numpy as np
from tqdm import tqdm

if __name__ == "__main__":
    # ----------------------------
    # Configuration and Setup
    # ----------------------------

    db_path = '/Users/martinli/Desktop/CancerGeneBot/Notebook/variants_fixed.db'
    csv_directory = '/Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/Annotated'
    output_directory = '/Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/AnnotatedClean_ReAnnoted'
    os.makedirs(output_directory, exist_ok=True)

    SQLITE_MAX_VARIABLE_NUMBER = 999
    max_positions_per_chunk = SQLITE_MAX_VARIABLE_NUMBER // 2  # (chrom, pos)

    # ----------------------------
    # Helpers
    # ----------------------------
    def is_valid_rsID(rsID):
        return isinstance(rsID, str) and rsID.startswith('rs')

    def normalize_chr(x: str) -> str:
        s = str(x).strip()
        if s.lower().startswith('chr'):
            s = s[3:]
        s = s.upper()
        mapping = {'M': 'MT', 'MT': 'MT', '23': 'X', '24': 'Y'}
        return mapping.get(s, s)

    def parse_pos_to_Int64(series: pd.Series) -> pd.Series:
        # Coerce to numeric, drop inf/-inf, allow only integer-valued numbers
        num = pd.to_numeric(series, errors='coerce')
        num = num.mask(~np.isfinite(num))               # inf/-inf -> NaN
        # keep values like 12345.0 but reject 12345.7
        num = num.where(num.isna() | ((num % 1) == 0), other=pd.NA)
        return num.astype('Int64')

    def query_positions(chunk, cursor):
        # chunk: list of (chrom_str, pos_int)
        placeholders = ', '.join(['(?, ?)'] * len(chunk))
        query = f'''
            SELECT chrom, pos, id
            FROM variants
            WHERE (chrom, pos) IN ({placeholders})
        '''
        params = [item for pair in chunk for item in pair]
        cursor.execute(query, params)
        rows = cursor.fetchall()
        # Key the map by (chrom as str, pos as int)
        return {(str(r[0]), int(r[1])): r[2] for r in rows}

    # ----------------------------
    # Connect to the Database
    # ----------------------------
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database not found at path: {db_path}")

    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('CREATE INDEX IF NOT EXISTS idx_chrom_pos ON variants (chrom, pos)')

    # ----------------------------
    # Process Each CSV File
    # ----------------------------
    csv_files = glob.glob(os.path.join(csv_directory, '*.csv'))
    print(f"Found {len(csv_files)} CSV files in directory: {csv_directory}")

    for csv_file_path in csv_files:
        print(f"\nProcessing file: {os.path.basename(csv_file_path)}")

        df = pd.read_csv(csv_file_path)

        required_columns = {'hm_rsID', 'hm_chr', 'hm_pos'}
        if not required_columns.issubset(df.columns):
            print(f"Error: Required columns missing in {os.path.basename(csv_file_path)}")
            # still save an untouched copy for traceability
            df.to_csv(os.path.join(output_directory, os.path.basename(csv_file_path)), index=False)
            continue

        # Normalize/parse without forcing entire column to plain int
        df['hm_chr'] = df['hm_chr'].astype(str).str.strip()
        df['hm_chr_norm'] = df['hm_chr'].apply(normalize_chr)

        df['hm_pos_int'] = parse_pos_to_Int64(df['hm_pos'])

        # Count invalid pos entries (NaN after parsing)
        invalid_pos_count = df['hm_pos_int'].isna().sum()
        if invalid_pos_count:
            print(f"Rows with invalid hm_pos (cannot be parsed to integer): {invalid_pos_count}")

        # Identify invalid hm_rsID
        invalid_rsID_mask = ~df['hm_rsID'].apply(is_valid_rsID)
        num_invalid_before = int(invalid_rsID_mask.sum())
        print(f"Number of invalid hm_rsID before imputation: {num_invalid_before}")

        if num_invalid_before == 0:
            print("No invalid hm_rsID found. Skipping imputation.")
            out_path = os.path.join(output_directory, os.path.basename(csv_file_path))
            df.drop(columns=['hm_chr_norm', 'hm_pos_int'], errors='ignore').to_csv(out_path, index=False)
            continue

        # Only query for rows where (chr,pos) are usable
        valid_site_mask = df['hm_chr_norm'].ne('') & df['hm_pos_int'].notna()
        to_query = df.loc[invalid_rsID_mask & valid_site_mask, ['hm_chr_norm', 'hm_pos_int']].drop_duplicates()

        position_to_rsID = {}
        if not to_query.empty:
            positions = [(chrom, int(pos)) for chrom, pos in to_query.itertuples(index=False, name=None)]
            for i in tqdm(range(0, len(positions), max_positions_per_chunk), desc="Querying database"):
                chunk = positions[i:i+max_positions_per_chunk]
                if not chunk:
                    continue
                mapping = query_positions(chunk, c)
                position_to_rsID.update(mapping)
        else:
            print("No queryable (chr, pos) pairs among invalid rsIDs.")

        # Impute
        def impute_rsID(row):
            if not is_valid_rsID(row['hm_rsID']):
                if pd.notna(row['hm_pos_int']):
                    key = (str(row['hm_chr_norm']), int(row['hm_pos_int']))
                    return position_to_rsID.get(key, row['hm_rsID'])
            return row['hm_rsID']

        df['hm_rsID'] = df.apply(impute_rsID, axis=1)

        num_invalid_after = int((~df['hm_rsID'].apply(is_valid_rsID)).sum())
        print(f"Number of invalid hm_rsID after imputation: {num_invalid_after}")

        # Save updated file
        output_file_name = os.path.basename(csv_file_path).replace('.csv', '_updated.csv')
        output_file_path = os.path.join(output_directory, output_file_name)
        df.drop(columns=['hm_chr_norm', 'hm_pos_int'], errors='ignore').to_csv(output_file_path, index=False)
        print(f"Updated file saved to: {output_file_path}")

    # ----------------------------
    # Cleanup
    # ----------------------------
    conn.close()
    print("\nProcessing of all CSV files completed.")

Found 31 CSV files in directory: /Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/Annotated

Processing file: PGS000393_annotated_dataset.csv
Rows with invalid hm_pos (cannot be parsed to integer): 1
Number of invalid hm_rsID before imputation: 1110379


Querying database: 100%|███████████████████████████████████████████████████████████| 2226/2226 [02:37<00:00, 14.18it/s]


Number of invalid hm_rsID after imputation: 535830
Updated file saved to: /Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/AnnotatedClean_ReAnnoted/PGS000393_annotated_dataset_updated.csv

Processing file: PGS004325_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS004512_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS003392_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 90


Querying database: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 49.76it/s]


Number of invalid hm_rsID after imputation: 29
Updated file saved to: /Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/AnnotatedClean_ReAnnoted/PGS003392_annotated_dataset_updated.csv

Processing file: PGS000391_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 14


...

Rows with invalid hm_pos (cannot be parsed to integer): 1
Number of invalid hm_rsID before imputation: 91


Querying database: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 173.58it/s]


Number of invalid hm_rsID after imputation: 52
Updated file saved to: /Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/AnnotatedClean_ReAnnoted/PGS004860_annotated_dataset_updated.csv

Processing file: PGS005163_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS000078_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS002270_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS004165_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS000880_annotated_dataset.csv
Number of invalid hm_rsID before imputation: 0
No invalid hm_rsID found. Skipping imputation.

Processing file: PGS000394_annotated_dataset.csv
Rows with invalid hm_pos (cannot be pa

Querying database: 100%|███████████████████████████████████████████████████████████| 2226/2226 [02:40<00:00, 13.85it/s]


Number of invalid hm_rsID after imputation: 535830
Updated file saved to: /Users/martinli/Desktop/CancerGeneBot/Notebook/CancerPGS/AnnotatedClean_ReAnnoted/PGS000394_annotated_dataset_updated.csv

Processing of all CSV files completed.
